In [39]:
%env KERAS_BACKEND=tensorflow
%matplotlib inline
import tensorflow as tf
import os
from keras.backend import set_session
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam, Adadelta

env: KERAS_BACKEND=tensorflow


In [40]:
#啟用GPU跑程式
def create_session(gpu_id='0', pp_mem_frac=None):

    tf.reset_default_graph()
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id # can multiple?
    with tf.device('/gpu:' + gpu_id):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        if pp_mem_frac is not None:
            config.gpu_options.per_process_gpu_memory_fraction=pp_mem_frac
        session = tf.Session(config = config)
    return session
gpu_id = '0'
sess = create_session(gpu_id)
set_session(sess)


**<font size = 5>讀取檔案+把資料reshape到可以放進cnn</font>**

In [41]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
# normalize
x_train=x_train/255
x_test=x_test/255

**<font size = 5>把label改成one hot編碼</font>**

In [42]:
y_train=np_utils.to_categorical(y_train,10)
y_test=np_utils.to_categorical(y_test,10)

**<font size = 5>建立模型</font>**

In [43]:
model = Sequential()
model.add(Conv2D(4,(3,3),padding='same',input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(8,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(16,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(15))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.004),metrics=['accuracy'])
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 4)         40        
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 4)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 4)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         296       
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 8)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 16)          1168      
__________

In [44]:
model.fit(x_train, y_train,validation_split=0.1, epochs=15,batch_size = 100)

Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 3s 51us/step - loss: 0.3185 - acc: 0.8974 - val_loss: 0.1025 - val_acc: 0.9703
Epoch 2/15
54000/54000 [==============================] - 2s 45us/step - loss: 0.0972 - acc: 0.9698 - val_loss: 0.0732 - val_acc: 0.9795
Epoch 3/15
54000/54000 [==============================] - 2s 46us/step - loss: 0.0773 - acc: 0.9759 - val_loss: 0.0783 - val_acc: 0.9763
Epoch 4/15
54000/54000 [==============================] - 2s 45us/step - loss: 0.0650 - acc: 0.9795 - val_loss: 0.0608 - val_acc: 0.9828
Epoch 5/15
54000/54000 [==============================] - 2s 46us/step - loss: 0.0592 - acc: 0.9815 - val_loss: 0.0610 - val_acc: 0.9815
Epoch 6/15
54000/54000 [==============================] - 2s 45us/step - loss: 0.0553 - acc: 0.9826 - val_loss: 0.0616 - val_acc: 0.9815
Epoch 7/15
54000/54000 [==============================] - 2s 45us/step - loss: 0.0499 - acc: 0.9841 - val_loss: 0.0485 - val_acc:

In [45]:
score = model.evaluate(x_test, y_test)
print("loss", score[0])
print("accuracy", score[1])

10000/10000 [==============================] - 0s 42us/step
loss 0.044683795692841526
accuracy 0.986
